# VWM project: analyse and parse the anime dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("animesWorking.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         19311 non-null  int64  
 1   title       19311 non-null  object 
 2   synopsis    18336 non-null  object 
 3   genre       19311 non-null  object 
 4   aired       19311 non-null  object 
 5   episodes    18605 non-null  float64
 6   members     19311 non-null  int64  
 7   popularity  19311 non-null  int64  
 8   ranked      16099 non-null  float64
 9   score       18732 non-null  float64
 10  img_url     19131 non-null  object 
 11  link        19311 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 1.8+ MB


In [4]:
df.head(10)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
5,37510,Mob Psycho 100 II,"Shigeo ""Mob"" Kageyama is now maturing and unde...","['Action', 'Slice of Life', 'Comedy', 'Superna...","Jan 7, 2019 to Apr 1, 2019",13.0,442310,176,21.0,8.89,https://cdn.myanimelist.net/images/anime/1918/...,https://myanimelist.net/anime/37510/Mob_Psycho...
6,199,Sen to Chihiro no Kamikakushi,"Stubborn, spoiled, and naïve, 10-year-old Chih...","['Adventure', 'Supernatural', 'Drama']","Jul 20, 2001",1.0,913212,40,20.0,8.90,https://cdn.myanimelist.net/images/anime/6/795...,https://myanimelist.net/anime/199/Sen_to_Chihi...
7,38000,Kimetsu no Yaiba,"Ever since the death of his father, the burden...","['Action', 'Demons', 'Historical', 'Shounen', ...","Apr 6, 2019 to Sep 28, 2019",26.0,575037,106,19.0,8.92,https://cdn.myanimelist.net/images/anime/1286/...,https://myanimelist.net/anime/38000/Kimetsu_no...
8,35247,Owarimonogatari 2nd Season,Following an encounter with oddity specialist ...,"['Mystery', 'Comedy', 'Supernatural', 'Vampire']","Aug 12, 2017 to Aug 13, 2017",7.0,189944,573,18.0,8.93,https://cdn.myanimelist.net/images/anime/6/873...,https://myanimelist.net/anime/35247/Owarimonog...
9,2904,Code Geass: Hangyaku no Lelouch R2,"One year has passed since the Black Rebellion,...","['Action', 'Military', 'Sci-Fi', 'Super Power'...","Apr 6, 2008 to Sep 28, 2008",25.0,992196,27,17.0,8.93,https://cdn.myanimelist.net/images/anime/4/939...,https://myanimelist.net/anime/2904/Code_Geass_...


In [5]:
# Získat základní statistiky o datech (count, mean, std, min, max,..)
df.describe()

,uid,episodes,members,popularity,ranked,score
count,19311.000000,18605.000000,1.931100e+04,19311.000000,16099.000000,18732.000000
mean,19358.904096,11.460414,3.472609e+04,7720.830304,6866.524194,6.436107
std,14271.446515,47.950386,1.121772e+05,4676.786104,4390.018768,1.007941
min,1.000000,1.000000,2.500000e+01,1.000000,1.000000,1.250000
25%,4833.500000,1.000000,3.880000e+02,3725.000000,2895.500000,5.770000
50%,18327.000000,2.000000,2.389000e+03,7539.000000,6963.000000,6.410000
75%,33896.500000,12.000000,1.450150e+04,11613.000000,10601.500000,7.150000
max,40960.000000,3057.000000,1.871043e+06,16338.000000,14675.000000,9.230000


In [ ]:
# Vybereme pouze řádky (ne)obsahující missing values
df.isnull() == df.isna()
# isnull, isna, notnull, notna
#print(df.isnull().sum())
df.notnull().sum()

In [ ]:
# Názvy sloupců v DataFrame
df.columns

In [ ]:
# Názvy řádků v DataFrame
df.index

#### Přístup ke sloupcům v DataFrame
- Přístup možný 2 způsoby:
    - `df["Age"]`
    - `df.Age`
- Po výběru jednoho sloupce již pracujeme s `pd.Series`!
- Pro tvorbu nového sloupce je nutné použít hranaté závorky
- Attribute access občas nejde použít (speciální název sloupce, mezery,..)

### Základy přístupu k datům 

In [6]:
# Vybereme sloupec Age (výsledkem je Series). Na ní vybereme prvních 10 hodnot.
col = df.genre

0        ['Comedy', 'Sports', 'Drama', 'School', 'Shoun...
1        ['Drama', 'Music', 'Romance', 'School', 'Shoun...
2        ['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...
3        ['Action', 'Military', 'Adventure', 'Comedy', ...
4         ['Action', 'Mystery', 'Supernatural', 'Vampire']
                               ...                        
19306           ['Sci-Fi', 'Adventure', 'Comedy', 'Magic']
19307    ['Adventure', 'Fantasy', 'Magic', 'Martial Art...
19308                ['Mystery', 'Horror', 'Supernatural']
19309    ['Action', 'Adventure', 'Comedy', 'Historical'...
19310    ['Military', 'Sci-Fi', 'Space', 'Drama', 'Mecha']
Name: genre, Length: 19311, dtype: object

In [ ]:
# Vybereme sloupce "Age" a "Sex". Výsledekem je DataFrame!
#df[["Age","Sex"]]

In [ ]:
# Jak vybrat 3 a 4 řádek dataframe?
df.iloc[3:5]

In [ ]:
# Kolik lidí bylo starších 30 let?
(df.Age>30).sum()

In [ ]:
# Kdo bydlel v kabině D33?
df[df.Cabin == "D33"]

In [ ]:
# Jak nalézt všechny jejichž jméno začíná na Goodwin?
df[df.Name.str.startswith('Goodwin')]

## Úloha 01: Zřetězení dat

  - Připojte data2.csv za data1.csv následujícím způsobem:
      - Data (sloupce), která nejsou v data1.csv jsou v data2.csv vynechána.
      - Spočítejte věk pomocí sloupce BirthYear (rok narození) v data2.csv a uložte jej do sloupce Age.
      - PasangerId musí být unikátní. Nastavíme ho jako nový index.

In [ ]:
# Načteme soubor data2.csv a zkontrolujeme, že sedí názvy sloupců a zbytečné odstraníme
df2 = ...

In [ ]:
# V nově načteném df spočítáme věk cestujících


In [ ]:
# Odstraníme zbytečné sloupce


In [ ]:
# Spojíme soubory data1.csv a data2.csv (existuje více způsobů). Všimněte si, že index nyní neodpovídá!


In [ ]:
# Dává smysl, abychom použili PassengerID jako nový index. Ovšem obsahuje neunikátní hodnoty! Let's fix that.


In [ ]:
# Nastavíme sloupec PassengerID jako nový index


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Matplotlib umožňuje změnit styl na jiný, než defaultní
matplotlib.style.use('ggplot')

# Bez násl. řádku někdy nefunguje vykreslování grafů v Jupyter noteboocích
%matplotlib inline 

### Základní chování grafů

In [ ]:
# Co se stane když pustíme graf bez žádných parametrů?
df.plot()
#df.info()
plt.show()

### Běžně používané typy grafů
* Ukážeme si běžně používané typy grafů na věku pasažérů

In [ ]:
# Line chart
df.Age.plot()
plt.show()

In [ ]:
# Histogram. Správný graf pro tento usecase.
df.Age.plot(kind='hist', title="My title")
plt.show()

In [ ]:
# Boxplot je taktéž vynikající graf
plt.figure(figsize=(6,6))
#df.boxplot()
df.boxplot(column=["Age"])
plt.show()

### Vizualizace závislosti na přežití

* Nejprve rozdělíme na dvě části - `survived` a `not_survived`

In [ ]:
import seaborn as sns
fig, ax = plt.subplots()
df.groupby(by='Pclass').Age.apply(lambda x: sns.distplot(x, label=x.name))
ax.set_xlabel("Age of passenger")
ax.legend()
plt.show()

In [ ]:
# Rozdělíme na survived a not_survived
survived = df[df['Survived'] == 1]
not_survived = df[df['Survived'] == 0]

* Podíváme se na závislost věku a třídy na přežití
    * Vzhledem k tomu, že se jedná o dvě proměné je potřeba dvourozměrný graf
    * Ideální bude scatter plot (ještě lepší by byl boxplot!)

In [ ]:
# df.boxplot nevyžaduje ani groupby
ax = df.boxplot(column='Age', by='Survived')
ax.set_title("")
ax.set_ylabel(" age")
plt.show()

In [ ]:
# Vykreslíme do dvou scatter grafů
survived.plot.scatter(x='Age', y='Pclass',color='Green',label='Survived', alpha=0.4)
plt.yticks([1,2,3])
not_survived.plot(x='Age', y='Pclass',kind='scatter', color='Black',label='Not Survived', alpha=0.4)
plt.yticks([1,2,3])
plt.show()

In [ ]:
# Lze vykreslit i do jednoho grafu (nutné předat axis objekt)


### Zobrazování subgrafů

* Co když chceme mí více grafů vedle sebe nebo i pod sebou?
* Dvě hlavní možnosti
    1. Metoda `plt.subplot`
    2. Metoda `plt.subplots`

In [ ]:
# Nastavíme velikost grafu (příliš malý graf je nečitelný)
plt.figure(figsize=(9,12)) # velikost figsize se určuje v palcích (angl. inches)

# První použijeme metodu plt.subplot
plt.subplot(321) # tři řádky a dva sloupce, přiřaď následující graf do prvního slotu   TOHLE JE DULEZITY RADEK
survived['Age'].plot.hist(color='Green')
plt.subplot(322)
not_survived['Age'].plot.hist(color='Black')
plt.subplot(323)
survived['Pclass'].plot.hist(color='Green')
plt.subplot(324)
not_survived['Pclass'].plot.hist(color='Black')
plt.subplot(325)
survived['Sex'].apply(lambda x: 1 if x == 'female' else 0).plot.hist(color='Green')
plt.subplot(326)
not_survived['Sex'].apply(lambda x: 1 if x == 'female' else 0).plot.hist(color='Black')

plt.show()

In [ ]:
# Druhá možnost je metoda plt.subplots
fig, ax = plt.subplots(3, 2, figsize=(9,12))

survived['Age'].plot.hist(color='Green', ax=ax[0,0])
not_survived['Age'].plot.hist(color='Black', ax=ax[0,1])

survived['Pclass'].plot.hist(color='Green', ax=ax[1,0])
not_survived['Pclass'].plot.hist(color='Black', ax=ax[1,1])

survived['Sex'].apply(lambda x: 1 if x == 'female' else 0).plot.hist(color='Green', ax=ax[2,0])
not_survived['Sex'].apply(lambda x: 1 if x == 'female' else 0).plot.hist(color='Black', ax=ax[2,1])

plt.show()

## Seaborne
* Využívá matplotlib
* Lze v ní snadno vytvořit grafy, které není snadné v matplotlibu napsat rychle
### Jak zjistit jaké mají příznaky vztahy mezi sebou?
* Chceme zjistit jak moc navzájem korelují jednotlivé příznaky
* Ideální graf na toto je heatmapa korelační matice

In [ ]:
# Heatmapa umí pracovat jen s číselnými hodnotami, je tedy nutné převést text na číselnou hodnotu
df.Sex = df.Sex.replace({"male": 0, "female": 1})

In [ ]:
# Vytvoříme korelační matici
cor_matrix = df.corr()
cor_matrix
# Zobrazení heatmapy
plt.figure(figsize=(14,12))
sns.heatmap(cor_matrix,annot=True)
plt.show()

In [ ]:
# Stripplot podle x=Pclass a y=Age, hue=Survived, data=df
plt.figure(figsize=(12,4))
sns.stripplot(x="Pclass", y="Age", hue="Survived", data=df,jitter=0.1, palette=['black','green'])
plt.show()


## Úloha 02: vytvořte bodový graf (scatter plot) pro všechny dvojice příznaků

  - K vykreslení všech (smysluplných) dvojic příznaků použijte metodu sns.pairplot, podobným způsobem jako metodu níže.

In [ ]:
# Použije Seaborn pairplot s tím, že nás zajímá přežití a chceme vidět histogramy na diagonále
sns.pairplot(df, hue="Survived", diag_kind="hist")
plt.show()

## Stahování dat z webu pomocí Pythonu a pandas (1. úkol)
          
### Tipy pro scrapping s Pythonem:
   - K získání HTML zdrojáku stránky s `url` použijte `import requests`: 
      - `r = requests.get(url)`
      - `html = r.text`
   - Metoda `pandas.read_html(r.text)` uloží všechny tabulky `<table>` jako seznam pandas DataFrames:
      - `list_of_data_frames = pd.read_html(html,flavor='html5lib')`
   - Pro HTML parsování lze použít `from bs4 import BeautifulSoup`.

In [ ]:
import requests

In [ ]:
# příklad pro statutární město Kladno, výsledky voleb pro rok 2010
url = 'https://www.volby.cz/pls/kv2010/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=2103&xobec=532053&xstat=0&xvyber=0'
dfs = pd.read_html(url,flavor='html5lib')

### Pro zajímavost (z předloňského roku): Příklad jednoduchého stažení dat z webu pomocí POST formuláře.

Úkolem je stáhnout všechna data z http://kap.ujak.cz/index.php a uložit je ve formátu pandas DataFrame.
#### První ukážeme variantu s pd.read_html

In [ ]:
# url s formulářem
url = 'https://kap.ujak.cz/index.php'
# nastavení POST proměnných simulující odeslání formuláře
data = {
    'prace' : 'BP', # DP = diplomka, DR = disertace, RI = rigorozní
    'nazev' : '%%%', # alespoň tři písmena z názvu hledané práce
    'pocet' : '0',
    'klic' : '', # alespoň tři písmena z klíčových slov
    'kl' : 'c', # c = částečně odpovídá, n = plně odpovídá
    'hledat' : 'Vyhledat'
}

data_all = pd.DataFrame()
for prace in ['BP', 'DP']:
    data['prace'] = prace
    r = requests.post(url, data=data)
    r.encoding='cp1250'
    ldf = pd.read_html(r.text,flavor='html5lib', header=0)
    df_ujak = ldf[0]
    strana = 30
    if data_all.shape[0] == 0:
        data_all = df.copy()
    else:
        data_all = pd.concat([data_all,df], ignore_index=True)
    while df.shape[0] > 0:
        if data_all.shape[0] > 200: # just to prevent from downloading all data
            break
        r = requests.post(url.format(strana), data)
        r.encoding='cp1250'
        ldf = pd.read_html(r.text,flavor='html5lib', header=0)
        df_ujak = ldf[0]
        strana = strana + 30
        data_all = pd.concat([data_all,df_ujak], ignore_index=True)

In [ ]:
data_all

#### Lze použít i requests a bs4
* Pokud se podíváme na původní stránku, můžeme si všimnout že název práce zároveň slouží jako odkaz na práci samotnou.
* Tento odkaz ale `pandas.read_html` nedokázalo extrahovat. 
* Zkusíme jej tedy extrahovat pomocí Beautiful Soup z bs4.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# url s formulářem
url = 'https://kap.ujak.cz/index.php'
# nastavení POST proměnných simulující odeslání formuláře
data = {
    'prace' : 'BP', # DP = diplomka, DR = disertace, RI = rigorozní
    'nazev' : '%%%', # alespoň tři písmena z názvu hledané práce
    'pocet' : '0',
    'klic' : '', # alespoň tři písmena z klíčových slov
    'kl' : 'c', # c = částečně odpovídá, n = plně odpovídá
    'hledat' : 'Vyhledat'
}

links = {'BP': [],
         'DP': []}

for prace in ['BP', 'DP']:
    data['prace'] = prace
    r = requests.post(url, data=data)
    r.encoding='cp1250'

    soup = BeautifulSoup(r.text, 'html.parser')  # Použije html parser na rozparsování stránky
    for line in soup.table.find_all('tr'):  # Z tabulky dostaneme všechny řádky
        links[prace].append(line.find('a').get('href'))  # Z každé řádky dostaneme odkaz na práci
    
    while len(links[prace]) > 0:  # Zpracování následujících stránek
        if len(links[prace]) > 200: # just to prevent from downloading all data
            break
        r = requests.post(url.format(strana), data)
        r.encoding='cp1250'
        soup = BeautifulSoup(r.text, 'html.parser')
        for line in soup.table.find_all('tr'):
            links[prace].append(line.find('a').get('href'))

In [ ]:
links_to_thesis = pd.DataFrame(links)
links_to_thesis

#### Další možností je použít přímo knihovnu pro scrapování
* Ukážeme si knihovnu `scrapy`
* Ve složce je soubor s názvem `ujak_crawler.py`
    * Tento script stáhne data o pracích i s linky na ně
    * Bohužel se špatně pouští v Jupyter Notebooku a tak jí pustíme přes shell

In [ ]:
!scrapy runspider crawler/ujak_crawler.py -o crawler/crawled_data.csv

### Pro zajímavost se můžeme podívat kolik prací se stejným názvem bylo na UJAK od roku 2000
* Tohoto se dá snadno docílit přes `groupby`

In [ ]:
# Načteme data z crawler/ujak.csv


In [ ]:
# Zjistíme počty prací se stejným názvem starších než v roce 2000 (filter, groupby, agregační operace)
